# This attack simply  inserts 'positive' words based on words commonly occuring in benign sentences at the beginning and end (but not in toxic sentences) 

- This is a white box attack since it assumes access to training data 

In [72]:
import pickle
import os 
import pandas as pd 
from collections import Counter

In [296]:
#tokenized training data looking at sentence 
path_to_training = r'../preprocessing/dump_2/train_sent.data'


#path to testing data we want to attack
exp_folder = r"../experiments/data/exp_4/"

target_path = os.path.join(exp_folder,"test_comments.csv")

#path to save adversarial examples
adv_folder = os.path.join(exp_folder,"adv")

#create if it does not already exist
if not os.path.isdir(adv_folder):
    os.mkdir(adv_folder)
    
adv_path = os.path.join(adv_folder,"test_comments.csv")

In [277]:
with open(path_to_training, "rb") as handle:
    sent_list, label_list = pickle.load(handle)

In [278]:
for i in range(5):
    print(sent_list[i],label_list[i][0])

['either', 'she', 'has', 'a', 'pig', 'nose', 'or', 'she', 'has', 'her', 'camera', 'at', 'a', 'bad', 'angle'] 1
['i', 'could', "n't", 'help', 'but', 'notice', 'that', 'i', 'could', 'see', 'up', 'her', 'nostrils', 'most', 'of', 'her', 'clips', 'lol'] 1
['you', 'ramble'] 0
['stop', 'speaking', 'hate', 'speak', 'land', 'walrus'] 1
['whore'] 1


In [230]:
#create a dataframe using sentence and labels

train_df = pd.DataFrame(columns=["sentence", "label"])

In [231]:
train_df["sentence"] = sent_list
train_df["label"] = [l[0] for l in label_list]

In [232]:
train_df

,sentence,label
0,"[either, she, has, a, pig, nose, or, she, has,...",1
1,"[i, could, n't, help, but, notice, that, i, co...",1
2,"[you, ramble]",0
3,"[stop, speaking, hate, speak, land, walrus]",1
4,[whore],1
...,...,...
21302,"[$mention$, what, doctor, willingly, removed, ...",0
21303,"[unless, you, did, n't, go, to, a, doctor, for...",0
21304,"[something, like, that, seems, like, it, would...",0
21305,[rip],0


In [233]:
train_df['label'].value_counts()/len(train_df)

0    0.833013
1    0.166987
Name: label, dtype: float64

In [234]:
corpus = list(train_df["sentence"].values)

In [235]:
corpus = [item for sublist in corpus for item in sublist]

In [236]:
corpus

['either',
 'she',
 'has',
 'a',
 'pig',
 'nose',
 'or',
 'she',
 'has',
 'her',
 'camera',
 'at',
 'a',
 'bad',
 'angle',
 'i',
 'could',
 "n't",
 'help',
 'but',
 'notice',
 'that',
 'i',
 'could',
 'see',
 'up',
 'her',
 'nostrils',
 'most',
 'of',
 'her',
 'clips',
 'lol',
 'you',
 'ramble',
 'stop',
 'speaking',
 'hate',
 'speak',
 'land',
 'walrus',
 'whore',
 '$mention$',
 'you',
 'do',
 "n't",
 'have',
 'to',
 'be',
 'religious',
 ',',
 'if',
 'she',
 'make',
 'some',
 'video',
 'about',
 'is',
 ',',
 'she',
 "'s",
 'screwed',
 '!',
 'im',
 'ok',
 'with',
 'that',
 '$smiley$',
 'hello',
 'diana',
 ',',
 'i',
 'enjoy',
 'your',
 'presentations',
 'they',
 'seem',
 'to',
 'offer',
 'insightful',
 'awareness',
 'i',
 'have',
 'been',
 'married',
 'to',
 'the',
 'woman',
 'who',
 'i',
 'love',
 'for',
 '$number$',
 'years',
 'she',
 'has',
 'her',
 'own',
 'problems',
 'and',
 'failures',
 'god',
 'knows',
 ',',
 'so',
 'do',
 'i',
 'i',
 'have',
 'not',
 'accepted',
 'offers',
 't

In [237]:
all_counts = Counter(corpus)

In [238]:
pos_sentences = train_df[train_df['label']==0]

In [239]:
pos_sentences

,sentence,label
2,"[you, ramble]",0
5,"[$mention$, you, do, n't, have, to, be, religi...",0
6,"[im, ok, with, that, $smiley$]",0
7,"[hello, diana, ,, i, enjoy, your, presentations]",0
8,"[they, seem, to, offer, insightful, awareness]",0
...,...,...
21301,"[many, women, does, the, same, against, the, f...",0
21302,"[$mention$, what, doctor, willingly, removed, ...",0
21303,"[unless, you, did, n't, go, to, a, doctor, for...",0
21304,"[something, like, that, seems, like, it, would...",0


In [240]:
#count the number of times each word occurs in positive sentences
pos_corpus = list(pos_sentences["sentence"].values)

In [241]:
pos_corpus

[['you', 'ramble'],
 ['$mention$',
  'you',
  'do',
  "n't",
  'have',
  'to',
  'be',
  'religious',
  ',',
  'if',
  'she',
  'make',
  'some',
  'video',
  'about',
  'is',
  ',',
  'she',
  "'s",
  'screwed',
  '!'],
 ['im', 'ok', 'with', 'that', '$smiley$'],
 ['hello', 'diana', ',', 'i', 'enjoy', 'your', 'presentations'],
 ['they', 'seem', 'to', 'offer', 'insightful', 'awareness'],
 ['i',
  'have',
  'been',
  'married',
  'to',
  'the',
  'woman',
  'who',
  'i',
  'love',
  'for',
  '$number$',
  'years'],
 ['she', 'has', 'her', 'own', 'problems', 'and', 'failures'],
 ['god', 'knows', ',', 'so', 'do', 'i'],
 ['i',
  'have',
  'not',
  'accepted',
  'offers',
  'to',
  'replace',
  'my',
  'wife',
  'with',
  'younger',
  ',',
  'prettier',
  'and',
  'even',
  'nicer',
  'women',
  'because',
  'i',
  'am',
  'devoted',
  'to',
  'my',
  'wife',
  'with',
  'all',
  'of',
  'my',
  'heart'],
 ['as',
  'paster',
  'mark',
  'gungor',
  'discusses',
  ',',
  'it',
  'makes',
  'fo

In [242]:
pos_corpus = [item for sublist in pos_corpus for item in sublist]

In [243]:
pos_corpus

['you',
 'ramble',
 '$mention$',
 'you',
 'do',
 "n't",
 'have',
 'to',
 'be',
 'religious',
 ',',
 'if',
 'she',
 'make',
 'some',
 'video',
 'about',
 'is',
 ',',
 'she',
 "'s",
 'screwed',
 '!',
 'im',
 'ok',
 'with',
 'that',
 '$smiley$',
 'hello',
 'diana',
 ',',
 'i',
 'enjoy',
 'your',
 'presentations',
 'they',
 'seem',
 'to',
 'offer',
 'insightful',
 'awareness',
 'i',
 'have',
 'been',
 'married',
 'to',
 'the',
 'woman',
 'who',
 'i',
 'love',
 'for',
 '$number$',
 'years',
 'she',
 'has',
 'her',
 'own',
 'problems',
 'and',
 'failures',
 'god',
 'knows',
 ',',
 'so',
 'do',
 'i',
 'i',
 'have',
 'not',
 'accepted',
 'offers',
 'to',
 'replace',
 'my',
 'wife',
 'with',
 'younger',
 ',',
 'prettier',
 'and',
 'even',
 'nicer',
 'women',
 'because',
 'i',
 'am',
 'devoted',
 'to',
 'my',
 'wife',
 'with',
 'all',
 'of',
 'my',
 'heart',
 'as',
 'paster',
 'mark',
 'gungor',
 'discusses',
 ',',
 'it',
 'makes',
 'for',
 'a',
 'better',
 'marriage',
 'and',
 'a',
 'happier',


In [244]:
count_pos = Counter(pos_corpus)

In [245]:
toxic_sentences = train_df[train_df['label']==1]

In [246]:
toxic_sentences

,sentence,label
0,"[either, she, has, a, pig, nose, or, she, has,...",1
1,"[i, could, n't, help, but, notice, that, i, co...",1
3,"[stop, speaking, hate, speak, land, walrus]",1
4,[whore],1
18,"[that, one, sentence, should, be, considered, ...",1
...,...,...
21280,"[most, of, these, feminist, just, need, a, goo...",1
21281,"[oh, my, god, i, hate, what, we, had, become, ...",1
21290,"[almost, all, feminists, of, both, genders, ar...",1
21300,"[$mention$, milo, is, a, provocateur, who, tar...",1


In [247]:
#count the number of times each word occurs in positive sentences
toxic_corpus = list(toxic_sentences["sentence"].values)

In [248]:
toxic_corpus

[['either',
  'she',
  'has',
  'a',
  'pig',
  'nose',
  'or',
  'she',
  'has',
  'her',
  'camera',
  'at',
  'a',
  'bad',
  'angle'],
 ['i',
  'could',
  "n't",
  'help',
  'but',
  'notice',
  'that',
  'i',
  'could',
  'see',
  'up',
  'her',
  'nostrils',
  'most',
  'of',
  'her',
  'clips',
  'lol'],
 ['stop', 'speaking', 'hate', 'speak', 'land', 'walrus'],
 ['whore'],
 ['that',
  'one',
  'sentence',
  'should',
  'be',
  'considered',
  'in',
  'the',
  'form',
  'of',
  'a',
  'question',
  'to',
  'put',
  'forward',
  'the',
  'point',
  'that',
  'only',
  'a',
  'simpleton',
  'could',
  "n't",
  'understand',
  'the',
  'associations',
  'as',
  'they',
  'were',
  'used'],
 ['feminism',
  'and',
  'the',
  'massive',
  'government',
  'funding',
  'behind',
  'it',
  ',',
  'is',
  'the',
  'shield',
  'that',
  'the',
  'true',
  'evil',
  'of',
  'gynocentrism',
  'hides',
  'behind'],
 ['if',
  'i',
  'want',
  'to',
  'get',
  'to',
  'the',
  'systematic',
  'c

In [249]:
toxic_corpus = [item for sublist in toxic_corpus for item in sublist]

In [250]:
toxic_corpus

['either',
 'she',
 'has',
 'a',
 'pig',
 'nose',
 'or',
 'she',
 'has',
 'her',
 'camera',
 'at',
 'a',
 'bad',
 'angle',
 'i',
 'could',
 "n't",
 'help',
 'but',
 'notice',
 'that',
 'i',
 'could',
 'see',
 'up',
 'her',
 'nostrils',
 'most',
 'of',
 'her',
 'clips',
 'lol',
 'stop',
 'speaking',
 'hate',
 'speak',
 'land',
 'walrus',
 'whore',
 'that',
 'one',
 'sentence',
 'should',
 'be',
 'considered',
 'in',
 'the',
 'form',
 'of',
 'a',
 'question',
 'to',
 'put',
 'forward',
 'the',
 'point',
 'that',
 'only',
 'a',
 'simpleton',
 'could',
 "n't",
 'understand',
 'the',
 'associations',
 'as',
 'they',
 'were',
 'used',
 'feminism',
 'and',
 'the',
 'massive',
 'government',
 'funding',
 'behind',
 'it',
 ',',
 'is',
 'the',
 'shield',
 'that',
 'the',
 'true',
 'evil',
 'of',
 'gynocentrism',
 'hides',
 'behind',
 'if',
 'i',
 'want',
 'to',
 'get',
 'to',
 'the',
 'systematic',
 'causes',
 'of',
 'the',
 'problem',
 ',',
 'as',
 'you',
 'call',
 'it',
 ',',
 'i',
 'first',
 

In [251]:
count_toxic = Counter(toxic_corpus)

In [252]:
count_toxic.most_common(50)

[(',', 1862),
 ('the', 1504),
 ('a', 1387),
 ('you', 1338),
 ('to', 1158),
 ('and', 1121),
 ('i', 913),
 ('that', 837),
 ('is', 815),
 ('of', 783),
 ('are', 619),
 ('it', 573),
 ("'s", 531),
 ("n't", 503),
 ('!', 479),
 ('in', 474),
 ('they', 420),
 ('do', 382),
 ('your', 378),
 ('\\?', 366),
 ('for', 356),
 ('this', 344),
 ('like', 340),
 ('$mention$', 320),
 ('she', 319),
 ('with', 310),
 ('just', 284),
 ('be', 282),
 ('on', 269),
 ('have', 268),
 ('as', 266),
 ('but', 255),
 ('her', 254),
 ('so', 254),
 ('not', 253),
 ('fuck', 249),
 ('women', 238),
 ('if', 224),
 ('what', 224),
 ('all', 221),
 ('$number$', 218),
 ('or', 217),
 ('he', 216),
 ("'re", 210),
 ('who', 205),
 ('fucking', 200),
 ('me', 198),
 ('get', 194),
 ('men', 188),
 ('my', 181)]

In [253]:
count_pos.most_common(50)

[(',', 8533),
 ('the', 7661),
 ('to', 6024),
 ('a', 5403),
 ('i', 5357),
 ('and', 4954),
 ('you', 4785),
 ('is', 4175),
 ('that', 4171),
 ('of', 3979),
 ('it', 3604),
 ("n't", 2570),
 ('in', 2504),
 ("'s", 2438),
 ('\\?', 2257),
 ('are', 2116),
 ('do', 1957),
 ('!', 1926),
 ('not', 1923),
 ('for', 1750),
 ('$mention$', 1683),
 ('this', 1575),
 ('have', 1562),
 ('they', 1535),
 ('be', 1522),
 ('with', 1486),
 ('but', 1461),
 ('as', 1334),
 ('on', 1329),
 ('what', 1279),
 ('she', 1256),
 ('was', 1238),
 ('like', 1185),
 ('if', 1182),
 ('your', 1146),
 ('just', 1124),
 ('so', 1037),
 ('women', 1034),
 ('$number$', 1018),
 ('or', 1018),
 ('about', 1005),
 ('my', 976),
 ('he', 958),
 ('people', 940),
 ('all', 920),
 ('her', 891),
 ('me', 877),
 ('can', 814),
 ("i'm", 784),
 ('men', 783)]

In [254]:
all_counts.most_common(50)

[(',', 10395),
 ('the', 9165),
 ('to', 7182),
 ('a', 6790),
 ('i', 6270),
 ('you', 6123),
 ('and', 6075),
 ('that', 5008),
 ('is', 4990),
 ('of', 4762),
 ('it', 4177),
 ("n't", 3073),
 ('in', 2978),
 ("'s", 2969),
 ('are', 2735),
 ('\\?', 2623),
 ('!', 2405),
 ('do', 2339),
 ('not', 2176),
 ('for', 2106),
 ('$mention$', 2003),
 ('they', 1955),
 ('this', 1919),
 ('have', 1830),
 ('be', 1804),
 ('with', 1796),
 ('but', 1716),
 ('as', 1600),
 ('on', 1598),
 ('she', 1575),
 ('like', 1525),
 ('your', 1524),
 ('what', 1503),
 ('just', 1408),
 ('if', 1406),
 ('was', 1406),
 ('so', 1291),
 ('women', 1272),
 ('$number$', 1236),
 ('or', 1235),
 ('about', 1185),
 ('he', 1174),
 ('my', 1157),
 ('her', 1145),
 ('all', 1141),
 ('people', 1099),
 ('me', 1075),
 ('can', 985),
 ('men', 971),
 ('who', 931)]

In [255]:
count_df = pd.DataFrame(columns = ["word","pos_ratio","neg_ratio"])

In [256]:
count_df["word"] = list(all_counts.keys())

In [257]:
count_df["pos_ratio"] = count_df["word"].apply(lambda word: count_pos[word]/all_counts[word] )

In [258]:
count_df["neg_ratio"] = count_df["word"].apply(lambda word: count_toxic[word]/all_counts[word] )

In [259]:
count_df["count"] = count_df["word"].apply(lambda word: all_counts[word] )

In [260]:
#sort based on difference in ration
count_df["dif"] = count_df["pos_ratio"] - count_df["neg_ratio"]

In [261]:
count_df = count_df.sort_values(by = ["dif","count"] , ascending=False)

In [308]:
count_df[0:100]

,word,pos_ratio,neg_ratio,count,dif
10201,thee,1.0,0.0,39,1.0
1448,specific,1.0,0.0,31,1.0
320,games,1.0,0.0,28,1.0
4668,liked,1.0,0.0,28,1.0
2109,study,1.0,0.0,27,1.0
...,...,...,...,...,...
3488,private,1.0,0.0,13,1.0
3532,account,1.0,0.0,13,1.0
4487,stump,1.0,0.0,13,1.0
4951,anonymous,1.0,0.0,13,1.0


In [309]:
#these will be our positive words!
attack_words = count_df[0:100]["word"].tolist()

In [310]:
attack_words

['thee',
 'specific',
 'games',
 'liked',
 'study',
 'online',
 'message',
 'knowledge',
 'beginning',
 'earlier',
 'posted',
 'court',
 'members',
 'approval',
 'agreed',
 'tend',
 'accepted',
 'oxford',
 'belief',
 'mutilated',
 'source',
 'conservative',
 'appeal',
 'responsible',
 'procedure',
 'credit',
 'hero',
 'reported',
 'activism',
 'accused',
 'list',
 'links',
 'validation',
 'purpose',
 'pissed',
 'entirely',
 'brilliant',
 'janice',
 'cheers',
 'wives',
 'communists',
 'benefit',
 'cults',
 'differences',
 'topics',
 'regarding',
 'labor',
 'propaganda',
 'names',
 'thy',
 'growing',
 'edit',
 'threat',
 'bias',
 'forgotten',
 'seemed',
 'stream',
 'mass',
 'centipede',
 'deaths',
 'relevant',
 'shown',
 'service',
 'southern',
 'latham',
 'stance',
 'abram',
 'posters',
 'professional',
 'minute',
 'rule',
 'marry',
 'collective',
 'reference',
 'ghomeshi',
 'fiamengo',
 'direct',
 'subscribers',
 'familiar',
 '20',
 'de',
 'upper',
 'appreciate',
 'ourselves',
 'wise',

In [311]:
#load data we will be attacking

target = pd.read_csv(target_path)

In [312]:
target

,Comment #,Assigned to:,Sentence,Prosocial,Abusive
0,1,Lauren,@user06652 You can't be serious?,No,No
1,1,Lauren,"Just google ""United Nations"".",No,No
2,1,Lauren,You may learn something.,No,No
3,2,Lauren,Feminist = Misandrist,No,Yes
4,3,Lauren,Where the fuck do these fat cunts learn this s...,No,Yes
...,...,...,...,...,...
26367,13038,Tiana,I will donate and I would encourage others to ...,No,No
26368,13038,Tiana,Do anything it takes to destroy these stupid c...,No,Yes
26369,13039,Tiana,"""100 lkies"". Try 6k :D",No,No
26370,13040,Tiana,You bet your faggot ass im sending you money r...,No,Yes


In [313]:
import random
import math
def attack_row(row):
    """
    Attacks a single row of the dataframe to evade the toxicity detector by adding a fixed % of words by length at the beginning and
    end of the sentence. Returns the modified sentence
    """
    PERC_INS = 0.5
   
    sentence = row["Sentence"]

    
    #only attack abusive points
    if row["Abusive"]=="Yes":
        len_sent = len(sentence.split(" "))
        
        NUM_INS = min(len(attack_words),math.ceil(PERC_INS*len_sent))
        insert_first = random.sample(attack_words, NUM_INS)
        insert_last = random.sample(attack_words, NUM_INS)
        
        return " ".join(insert_first) + " " + sentence + " " + " ".join(insert_last)
    else:
        return sentence
        
        
    
    
    

In [314]:
attacked_target = target
    
    

In [315]:
attacked_target["Sentence"] = target.apply(attack_row, axis=1)

In [316]:
attacked_target

,Comment #,Assigned to:,Sentence,Prosocial,Abusive
0,1,Lauren,@user06652 You can't be serious?,No,No
1,1,Lauren,"Just google ""United Nations"".",No,No
2,1,Lauren,You may learn something.,No,No
3,2,Lauren,credit growing Feminist = Misandrist communist...,No,Yes
4,3,Lauren,benefit familiar private communists activism W...,No,Yes
...,...,...,...,...,...
26367,13038,Tiana,I will donate and I would encourage others to ...,No,No
26368,13038,Tiana,belief field shall sources tend Do anything it...,No,Yes
26369,13039,Tiana,"""100 lkies"". Try 6k :D",No,No
26370,13040,Tiana,janice throughout accusing wise list forgotten...,No,Yes


In [317]:
#save the attacked data to test!

attacked_target.to_csv(adv_path, index=False)

In [318]:
attacked_target.iloc[4]["Sentence"]

'benefit familiar private communists activism Where the fuck do these fat cunts learn this shit?ï»¿ student posting wives games de'

In [284]:
adv_path

'../experiments/data/exp_3/adv\\test_comments.csv'